In [122]:
import numpy as np
import pandas as pd

# DonChian Channel Formula  
# uc = upper channel
# n = number of minutes, hours, days, weeks, months
# period = minutes, hours, days, weeks, months
# lc = lower channel

In [123]:
df = pd.read_csv('data/BTCUSDT_data.csv')
df.drop(["Unnamed: 0", "index"], axis = 1, inplace = True)

In [124]:
# Average True Range (ATR) Formula
# n = number of periods
# tr = true range
# h = Today's high
# l = Today's low 
# c = Yesterday's closing price
# Max = Highest value of the three terms 
# hl = Today's high minus the low
# hc = Absolute value of today's high minus yesterday's closing price 
# lc = Absolute value of today's low minus yesterday's closing price 

In [125]:
def calculate_atr(data, period=14):
    high = data['High']
    low = data['Low']
    close = data['Close']
    atr = pd.Series(0.0, index=data.index)
    atr[0] = close[0]
    for i in range(1, len(close)):
        range1 = high[i] - low[i]
        range2 = abs(high[i] - close[i-1])
        range3 = abs(low[i] - close[i-1])
        true_range = max(range1, range2, range3)
        atr[i] = ((period - 1) * atr[i-1] + true_range) / period
    data['ATR'] = atr

    return data

# Calculate ATR using 14 periods
atr = calculate_atr(df, 14)
atr

,Name,Time,Open,High,Low,Close,Volume,ATR
0,BTCUSDT,2022-12-06 04:51:00,17004.99,17009.01,17002.34,17007.29,93.81524,17007.290000
1,BTCUSDT,2022-12-06 04:52:00,17007.55,17011.68,17005.52,17009.79,147.49372,15792.923571
2,BTCUSDT,2022-12-06 04:53:00,17009.79,17013.96,17007.55,17010.09,166.51496,14665.315459
3,BTCUSDT,2022-12-06 04:54:00,17010.00,17013.92,17009.43,17011.90,36.54084,13618.113641
4,BTCUSDT,2022-12-06 04:55:00,17011.90,17012.43,17009.36,17010.35,39.08232,12645.610523
...,...,...,...,...,...,...,...,...
83224,BTCUSDT,2023-02-01 23:55:00,23704.51,23711.98,23693.64,23707.48,123.78742,16.080049
83225,BTCUSDT,2023-02-01 23:56:00,23708.59,23719.69,23704.29,23714.49,157.83294,16.031474
83226,BTCUSDT,2023-02-01 23:57:00,23713.36,23726.83,23712.72,23722.29,170.78623,15.894226
83227,BTCUSDT,2023-02-01 23:58:00,23721.50,23736.12,23712.27,23716.12,254.51710,16.462495


In [126]:
import pandas as pd

def calculate_obv(data):
    close = data['Close']
    volume = data['Volume']
    obv = pd.Series(0, index=data.index)
    obv[0] = volume[0]
    for i in range(1, len(close)):
        if close[i] > close[i-1]:
            obv[i] = obv[i-1] + volume[i]
        elif close[i] < close[i-1]:
            obv[i] = obv[i-1] - volume[i]
        else:
            obv[i] = obv[i-1]
    return obv

obv = calculate_obv(df)
obv

,Name,Time,Open,High,Low,Close,Volume,ATR,OBV
0,BTCUSDT,2022-12-06 04:51:00,17004.99,17009.01,17002.34,17007.29,93.81524,17007.290000,93.81524
1,BTCUSDT,2022-12-06 04:52:00,17007.55,17011.68,17005.52,17009.79,147.49372,15792.923571,241.30896
2,BTCUSDT,2022-12-06 04:53:00,17009.79,17013.96,17007.55,17010.09,166.51496,14665.315459,407.82392
3,BTCUSDT,2022-12-06 04:54:00,17010.00,17013.92,17009.43,17011.90,36.54084,13618.113641,444.36476
4,BTCUSDT,2022-12-06 04:55:00,17011.90,17012.43,17009.36,17010.35,39.08232,12645.610523,405.28244
...,...,...,...,...,...,...,...,...,...
83224,BTCUSDT,2023-02-01 23:55:00,23704.51,23711.98,23693.64,23707.48,123.78742,16.080049,139041.69704
83225,BTCUSDT,2023-02-01 23:56:00,23708.59,23719.69,23704.29,23714.49,157.83294,16.031474,139199.52998
83226,BTCUSDT,2023-02-01 23:57:00,23713.36,23726.83,23712.72,23722.29,170.78623,15.894226,139370.31621
83227,BTCUSDT,2023-02-01 23:58:00,23721.50,23736.12,23712.27,23716.12,254.51710,16.462495,139115.79911


In [127]:
def calculate_donchian_channels(data, n):
    close = data['Close']
    high_n = close.rolling(window=n).max()
    low_n = close.rolling(window=n).min()
    mid_n = (high_n + low_n) / 2
    upper_channel = mid_n + (mid_n - low_n)
    lower_channel = mid_n - (high_n - mid_n)
    data['Upper Channel'] = upper_channel
    data['Lower Channel'] = lower_channel
    upper_channel_values = upper_channel.to_list()
    upper_channel_values.append(None)
    lower_channel_values = lower_channel.to_list()
    lower_channel_values.append(None)
    data['Upper Channel Values'] = upper_channel_values[1:]
    data['Lower Channel Values'] = lower_channel_values[1:]
    return data



donchian = calculate_donchian_channels(df, 20)
donchian


,Name,Time,Open,High,Low,Close,Volume,ATR,OBV,Upper Channel,Lower Channel,Upper Channel Values,Lower Channel Values
0,BTCUSDT,2022-12-06 04:51:00,17004.99,17009.01,17002.34,17007.29,93.81524,17007.290000,93.81524,NaN,NaN,NaN,NaN
1,BTCUSDT,2022-12-06 04:52:00,17007.55,17011.68,17005.52,17009.79,147.49372,15792.923571,241.30896,NaN,NaN,NaN,NaN
2,BTCUSDT,2022-12-06 04:53:00,17009.79,17013.96,17007.55,17010.09,166.51496,14665.315459,407.82392,NaN,NaN,NaN,NaN
3,BTCUSDT,2022-12-06 04:54:00,17010.00,17013.92,17009.43,17011.90,36.54084,13618.113641,444.36476,NaN,NaN,NaN,NaN
4,BTCUSDT,2022-12-06 04:55:00,17011.90,17012.43,17009.36,17010.35,39.08232,12645.610523,405.28244,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
83224,BTCUSDT,2023-02-01 23:55:00,23704.51,23711.98,23693.64,23707.48,123.78742,16.080049,139041.69704,23733.57,23694.87,23718.11,23694.87
83225,BTCUSDT,2023-02-01 23:56:00,23708.59,23719.69,23704.29,23714.49,157.83294,16.031474,139199.52998,23718.11,23694.87,23722.29,23694.87
83226,BTCUSDT,2023-02-01 23:57:00,23713.36,23726.83,23712.72,23722.29,170.78623,15.894226,139370.31621,23722.29,23694.87,23722.29,23694.87
83227,BTCUSDT,2023-02-01 23:58:00,23721.50,23736.12,23712.27,23716.12,254.51710,16.462495,139115.79911,23722.29,23694.87,23732.66,23694.87


In [128]:
h = df.iloc[-1, 4]
dataFrame = df.iloc[-1, :]
dataFrame

Name                                BTCUSDT
Time                    2023-02-01 23:59:00
Open                               23715.63
High                                23733.1
Low                                23710.86
Close                              23732.66
Volume                            137.08229
ATR                               16.875174
OBV                             139252.8814
Upper Channel                      23732.66
Lower Channel                      23694.87
Upper Channel Values                    NaN
Lower Channel Values                    NaN
Name: 83228, dtype: object

In [129]:
# On-Balance Volume Formula
# obv = current on-balance volume level
# pobv = previous on-balance volume level 
# volume = latest trading volume amount 

def onBalVol(obv, pobv, volume):
    return

df

,Name,Time,Open,High,Low,Close,Volume,ATR,OBV,Upper Channel,Lower Channel,Upper Channel Values,Lower Channel Values
0,BTCUSDT,2022-12-06 04:51:00,17004.99,17009.01,17002.34,17007.29,93.81524,17007.290000,93.81524,NaN,NaN,NaN,NaN
1,BTCUSDT,2022-12-06 04:52:00,17007.55,17011.68,17005.52,17009.79,147.49372,15792.923571,241.30896,NaN,NaN,NaN,NaN
2,BTCUSDT,2022-12-06 04:53:00,17009.79,17013.96,17007.55,17010.09,166.51496,14665.315459,407.82392,NaN,NaN,NaN,NaN
3,BTCUSDT,2022-12-06 04:54:00,17010.00,17013.92,17009.43,17011.90,36.54084,13618.113641,444.36476,NaN,NaN,NaN,NaN
4,BTCUSDT,2022-12-06 04:55:00,17011.90,17012.43,17009.36,17010.35,39.08232,12645.610523,405.28244,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
83224,BTCUSDT,2023-02-01 23:55:00,23704.51,23711.98,23693.64,23707.48,123.78742,16.080049,139041.69704,23733.57,23694.87,23718.11,23694.87
83225,BTCUSDT,2023-02-01 23:56:00,23708.59,23719.69,23704.29,23714.49,157.83294,16.031474,139199.52998,23718.11,23694.87,23722.29,23694.87
83226,BTCUSDT,2023-02-01 23:57:00,23713.36,23726.83,23712.72,23722.29,170.78623,15.894226,139370.31621,23722.29,23694.87,23722.29,23694.87
83227,BTCUSDT,2023-02-01 23:58:00,23721.50,23736.12,23712.27,23716.12,254.51710,16.462495,139115.79911,23722.29,23694.87,23732.66,23694.87
